In [0]:
from pyspark.sql.functions import col, to_timestamp, to_date, date_format, current_timestamp

# Comentário: Configuração de caminhos no S3
path_bronze = "s3://aws-data-lakehouse/bronze/crypto/"
path_silver = "s3://aws-data-lakehouse/silver/crypto/"

# Comentário: Leitura da camada Bronze
df_bronze = spark.read.format("delta").load(path_bronze)

# Comentário: Transformação técnica de tipos e limpeza de duplicatas
df_silver = df_bronze.select(
    "id",
    "name",
    "symbol",
    col("current_price").cast("double"),
    to_timestamp(col("last_updated")).alias("last_updated")
).withColumn(
    "last_updated_date", to_date(col("last_updated"))
).withColumn(
    "last_updated_time", date_format(col("last_updated"), "HH:mm:ss")
).withColumn("processed_to_silver", current_timestamp()
).dropDuplicates(["id", "last_updated"])

# Comentário: Escrita exclusiva na camada Silver em formato Delta
df_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(path_silver)

# Comentário: Confirmação da operação física
print(f"✅ Dados persistidos na Silver S3: {path_silver}")
display(df_silver.limit(10))